In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    pipeline, DataCollatorForSeq2Seq, EarlyStoppingCallback
)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# Read the CSV file
data = pd.read_csv("/kaggle/input/english-nagamese-raw-data/raw_data.csv")

# Verify the data
print(data.head())


                                             English  \
0  Paul, Silvanus, and Timothy to the church of t...   
1  We always give thanks to God for all of you as...   
2  We remember before our God and Father your wor...   
3  Brothers loved by God, we know he has chosen you,   
4  because our gospel came to you not in word onl...   

                                            Nagamese  
0  Paul aru Silvanus aru Timothy pora Isor aru Pr...  
1  Amikhan hodai apnikhan nimite Isor ke dhanyaba...  
2  Amikhan pora apni khan laga biswas laga kaam, ...  
3  Isor pora morom kora bhai khan, amikhan jane T...  
4  kilemane Isor laga kotha apni khan logote khal...  


In [3]:
# data=data[:100]

In [4]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [5]:
new_lang_token = "<ng>"
tokenizer.add_special_tokens(
    {"additional_special_tokens": [new_lang_token]},
    replace_additional_special_tokens=False
)
model.resize_token_embeddings(len(tokenizer))

M2M100ScaledWordEmbedding(256205, 1024, padding_idx=1)

In [6]:
source_texts = list(data["Nagamese"]) 
target_texts = list(data["English"])  




# Split the data into training and validation sets
train_source_texts, val_source_texts, train_target_texts, val_target_texts = train_test_split(
    source_texts, target_texts, test_size=0.20, random_state=40
)


In [7]:
from datasets import Dataset

# Tokenize function
def tokenize_function(examples):
    tokenizer.src_lang = "<ng>"
    tokenizer.tgt_lang = "<en>"
    model_inputs = tokenizer(examples["source_texts"], text_target=examples["target_texts"], return_tensors="pt",max_length=128, truncation=True, padding="max_length")
    return model_inputs

# Convert lists to Hugging Face Dataset
train_dataset = Dataset.from_dict({"source_texts": train_source_texts, "target_texts": train_target_texts})
val_dataset = Dataset.from_dict({"source_texts": val_source_texts, "target_texts": val_target_texts})

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/6360 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [8]:
print(train_dataset["input_ids"][0])
print(train_dataset["input_ids"][1])

print(train_dataset["labels"][0])
print(train_dataset["labels"][1])

print(tokenizer.convert_tokens_to_ids("<en>"))
print(tokenizer.convert_tokens_to_ids("<ng>"))

[256204, 65244, 8118, 100004, 469, 6207, 599, 330, 131, 28793, 71499, 181, 248079, 12361, 18181, 3480, 102, 262, 49811, 6207, 599, 117847, 54201, 9991, 197, 10559, 25, 150, 49811, 248079, 10687, 27008, 18181, 28, 1291, 3480, 102, 30532, 262, 49811, 6207, 599, 248075, 27266, 6207, 599, 248079, 9111, 264, 330, 61007, 8118, 6888, 248075, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[256204, 27266, 264, 7087, 1823, 808, 110773, 667, 8118, 2452, 5714, 22213, 248280, 48106, 105079, 653, 414, 66, 400, 221402, 248075, 27266, 264, 607, 12898, 54201, 2452, 5714, 130, 248280, 48106, 28725, 54, 414, 66, 400, 221402, 248075, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
!pip install transformers sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 40.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.0 MB/s eta 0:00:00


In [10]:
import sacrebleu

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode tokenized outputs into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # SacreBLEU expects reference translations as a **list of lists**
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])

    return {"bleu": bleu.score}

In [11]:
# # Define Data Collator
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps", 
    save_strategy="steps",
    eval_steps=200,
    save_steps=200,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    warmup_steps=200,
    seed = 40,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=25,
    predict_with_generate=True,
    optim="adafactor", 
    report_to=[],
    fp16=True,
    logging_dir="./logs",
    logging_steps=200,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=4)],
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)

# Start Training
trainer.train()


Step,Training Loss,Validation Loss,Bleu
200,7.593400,4.023510,2.868720
400,1.696700,0.414088,22.538401
600,0.328000,0.270752,36.326130
800,0.232300,0.248530,39.800972
1000,0.188900,0.243455,40.662911
1200,0.160300,0.243637,40.738594
1400,0.139000,0.248037,41.037590
1600,0.123400,0.254463,41.265640
1800,0.111200,0.257935,41.581871
2000,0.102100,0.262316,41.532529


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=2475, training_loss=0.8805530877546831, metrics={'train_runtime': 14881.7016, 'train_samples_per_second': 10.684, 'train_steps_per_second': 0.166, 'total_flos': 4.264861856956416e+16, 'train_loss': 0.8805530877546831, 'epoch': 24.753768844221106})

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [13]:
model = model.to(device)

In [14]:
source_texts = val_source_texts
reference_texts = val_target_texts

In [15]:
# Translate source texts using the model
model.eval()
machine_translations = []

for text in source_texts:
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids("<ng>"))
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    machine_translations.append(translated_text)

In [16]:

# Ensure reference_texts is in the required format
wrapped_reference_texts = [reference_texts] 

# Compute BLEU score
bleu = sacrebleu.corpus_bleu(machine_translations, wrapped_reference_texts)
print("BLEU Score:", bleu.score)

BLEU Score: 38.4846496926621
